## Determine which professionals Vanderbilt Hospital should reach out to in the Nashville area to expand their own patient volume.
## First, research which professionals are sending significant numbers of patients only to competitor hospitals (such as TriStar Centennial Medical Center).
## Next, consider the specialty of the provider. If Vanderbilt wants to increase volume from Orthopedic Surgeons or from Family Medicine doctors who should they reach out to in those areas?

In [2]:
import sqlite3
import pandas as pd
pd.set_option('display.max_rows', None)

### Produce a list of Hospitals with 'Vanderbilt' in the organization name in Nashville TN, not every organization from this list is in Hop table

In [49]:
query = '''
SELECT npi, `provider_organization_name_(legal_business_name)`, 
`provider_business_practice_location_address_city_name`, COUNT(*)
FROM nppes
WHERE `provider_organization_name_(legal_business_name)` LIKE '%Vanderbilt%' 
AND `provider_business_practice_location_address_state_name` = 'TN'
GROUP BY `provider_organization_name_(legal_business_name)`;
'''


with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    test_sqlite = pd.read_sql(query, db)
    
test_sqlite

# Not all organizations from this list appear in HOP dataset

,npi,provider_organization_name_(legal_business_name),provider_business_practice_location_address_city_name,COUNT(*)
0,1891951513,CONCENTRA VANDERBILT LLC,MURFREESBORO,5
1,1881885960,"DOUGLAS L. VANDERBILT, M.D. ,P.C.",CHATTANOOGA,1
2,1255577466,MONROE CARELL JR VANDERBILT CHILDREN'S HOSPITAL,NASHVILLE,1
3,1003211145,MONROE CARELL JR. CHILDREN'S HOSPITAL AT VANDE...,NASHVILLE,1
4,1578646006,VANDERBILT ASTHMA SINUS ALLERGY PROGRAM,NASHVILLE,1
5,1134306244,"VANDERBILT ASTHMA SINUS AND ALLERGY PROGRAM, LLC",NASHVILLE,1
6,1649860305,"VANDERBILT BEDFORD HOSPITAL, LLC",SHELBYVILLE,9
7,1609056399,VANDERBILT CHILDREN'S,NASHVILLE,1
8,1942757307,VANDERBILT CHILDREN'S HOSPITAL PHARMACY,NASHVILLE,1
9,1326224015,VANDERBILT CHILDRENS HOSPITAL,NASHVILLE,1


### We learned that 'VANDERBILT UNIVERSITY MEDICAL CENTER' has the most patient referrals from previous tasks. 
### Moreover, 'VANDERBILT UNIVERSITY MEDICAL CENTER' has MANY distinct NPI #. So the next step is to store all npi with organization name 'VANDERBILT UNIVERSITY MEDICAL CENTER' in a list

In [34]:
# Add 'OR `provider_organization_name_(legal_business_name)` = 'SHELBYVILLE HOSPITAL COMPANY LLC' in the WHERE clause if true
query = '''
SELECT GROUP_CONCAT(DISTINCT npi) AS Vanderbilt_npi
FROM nppes
WHERE `provider_organization_name_(legal_business_name)` = 'VANDERBILT UNIVERSITY MEDICAL CENTER'
AND `provider_business_practice_location_address_state_name` = 'TN'
'''


with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    npi_concat_sqlite = pd.read_sql(query, db)
    
Vanderbilt_npi = npi_concat_sqlite.iloc[0][0]
Vanderbilt_npi

'1477709053,1245485200,1366679276,1073833778,1528388196,1801161708,1508132242,1427394634,1336560598,1265833487,1487029997,1902261860,1659738383,1932352283,1912443326,1063958270,1700800877,1538670179,1679971485,1649637794,1013553155,1700950284,1972711828,1285091330,1548920465,1053449660,1144857160,1194186700,1215078027,1215979190,1306889597,1396882205,1558408633,1821030842,1104202761,1770944464,1972027159,1740319847'

## First, research which professionals are sending significant numbers of patients only to competitor hospitals (such as TriStar Centennial Medical Center).

In [50]:
# Get NONE Vandy providers
query = f'''
WITH Vandy_providers AS (
    SELECT DISTINCT hop.from_npi
    FROM hop_major_TN_hospitals AS hop
    WHERE hop.to_npi IN ({Vanderbilt_npi}))
SELECT hop.from_npi, SUM(patient_count) AS total_patients, n.*
FROM hop_major_TN_hospitals AS hop
JOIN nppes AS n
ON n.npi = hop.from_npi
WHERE hop.from_npi NOT IN (SELECT * FROM Vandy_providers)
GROUP BY hop.from_npi
ORDER BY total_patients DESC
LIMIT 100;
'''


with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    None_Vandy_sqlite = pd.read_sql(query, db)
None_Vandy_sqlite


,from_npi,total_patients,npi,entity_type_code,provider_organization_name_(legal_business_name),provider_last_name_(legal_name),provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,provider_credential_text,provider_first_line_business_practice_location_address,provider_second_line_business_practice_location_address,provider_business_practice_location_address_city_name,provider_business_practice_location_address_state_name,provider_business_practice_location_address_postal_code,primary_taxonomy
0,1417131715,6360,1417131715,1.0,None,RIDDICK,JOHN,ALSTON,DR.,None,M.D.,2400 PATTERSON ST,SUITE 502,NASHVILLE,TN,37203,207RI0011X
1,1912984758,4700,1912984758,1.0,None,LASSITER,GREGORY,L,DR.,None,MD,210 25TH AVE N STE 1204,None,NASHVILLE,TN,37203,2085R0202X
2,1871548818,4609,1871548818,1.0,None,KLEIN,WILLIAM,J,DR.,None,MD,7945 WOLF RIVER BLVD,None,GERMANTOWN,TN,38138,2085R0202X
3,1306993282,4334,1306993282,1.0,None,SHIPMAN,JASON,None,None,None,MD,210 25TH AVE N STE 1204,None,NASHVILLE,TN,37203,2085R0202X
4,1376756742,4082,1376756742,1.0,None,GRIFFIN,BENJAMIN,DAVID,None,None,M.D.,210 25TH AVE N STE 1204,None,NASHVILLE,TN,37203,2085R0202X
5,1447234141,3819,1447234141,1.0,None,CAPLAN,STUART,H,DR.,None,MD,210 25TH AVE N STE 1204,None,NASHVILLE,TN,37203,2085R0202X
6,1821060526,3797,1821060526,1.0,None,WATERS,RONALD,DOUGLAS,DR.,None,M.D.,210 25TH AVE N STE 1204,None,NASHVILLE,TN,37203,2085R0202X
7,1073531042,3768,1073531042,1.0,None,GRIPPO,JAMES,DAVID,DR.,None,M.D.,1224 TROTWOOD AVE,None,COLUMBIA,TN,38401,2085R0202X
8,1487816948,3752,1487816948,1.0,None,PATTERSON,GARY,TRAVIS,DR.,None,M.D.,210 25TH AVE N STE 1204,None,NASHVILLE,TN,37203,2085R0202X
9,1710932017,3735,1710932017,1.0,None,HIMMELFARB,ELLIOT,H,MR.,None,M.D.,3024 BUSINESS PARK CIR,None,GOODLETTSVILLE,TN,37072,2085R0202X


## Next, consider the specialty of the provider. If Vanderbilt wants to increase volume from Orthopedic Surgeons or from Family Medicine doctors who should they reach out to in those areas?

In [55]:
# Group NONE Vandy providers by specialization
query = f'''
WITH specialization AS (
    SELECT t.specialization, SUM(h.transaction_count) AS total_transactions, SUM(h.patient_count) AS total_patients
    FROM hop_major_TN_hospitals AS h
    INNER JOIN nppes n1
    ON h.from_npi = n1.npi
    INNER JOIN taxonomy as t
    ON n1.primary_taxonomy = t.code
    GROUP BY t.specialization
),

Vandy_providers AS (
    SELECT DISTINCT h.from_npi
    FROM hop_major_TN_hospitals AS h
    WHERE h.to_npi IN ({Vanderbilt_npi}))
    
SELECT t. specialization, SUM(patient_count) AS total_patients, 
       s.total_patients AS specialization_total_patients_ref,
       SUM(h.patient_count)*1.0/s.total_patients AS specialization_referral_ratio,
       n.*
FROM hop_major_TN_hospitals AS h
JOIN nppes AS n
ON n.npi = h.from_npi
JOIN taxonomy AS t
ON t.code = n.primary_taxonomy
INNER JOIN specialization AS s
ON s.specialization = t.specialization 
WHERE h.from_npi NOT IN (SELECT * FROM Vandy_providers)
GROUP BY h.from_npi
ORDER BY t.specialization, total_patients DESC ;
'''


with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    None_Vandy_byspecialization_sqlite = pd.read_sql(query, db)
None_Vandy_byspecialization_sqlite

,specialization,total_patients,specialization_total_patients_ref,specialization_referral_ratio,npi,entity_type_code,provider_organization_name_(legal_business_name),provider_last_name_(legal_name),provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,provider_credential_text,provider_first_line_business_practice_location_address,provider_second_line_business_practice_location_address,provider_business_practice_location_address_city_name,provider_business_practice_location_address_state_name,provider_business_practice_location_address_postal_code,primary_taxonomy
0,Acute Care,555,24337,0.022805,1649718230,1.0,None,GRABOVICH,DANIELLE,None,None,None,APN,1840 MEDICAL CENTER PKWY STE 201,None,MURFREESBORO,TN,37129,363LA2100X
1,Acute Care,526,24337,0.021613,1487041513,1.0,None,NEUFELD,LUCAS,DANIEL,None,None,AGACNP-BC,2300 PATTERSON ST,None,NASHVILLE,TN,37203,363LA2100X
2,Acute Care,388,24337,0.015943,1427334176,1.0,None,GRAUZER,FRANKLIN,WILLIAM,None,None,APN,4230 HARDING RD,SUITE 435,NASHVILLE,TN,37205,363LA2100X
3,Acute Care,347,24337,0.014258,1508892688,1.0,None,PEMBRIDGE,RICHARD,L,None,JR.,NP,2010 CHURCH ST,SUITE 201,NASHVILLE,TN,37203,363LA2100X
4,Acute Care,331,24337,0.013601,1811258213,1.0,None,HILL,KIMBERLY,WALDROP,MRS.,None,NP,4230 HARDING PIKE,SUITE 503,NASHVILLE,TN,37205,363LA2100X
5,Acute Care,328,24337,0.013477,1750747309,1.0,None,PINKSTON,JOSHUA,None,None,None,AGACNP-BC,920 DOUG WHITE DR STE 130,None,MYRTLE BEACH,SC,29572,363LA2100X
6,Acute Care,323,24337,0.013272,1568784205,1.0,None,MILAM,DANIEL,J,None,None,ACNP,4230 HARDING PIKE,SUITE 503,NASHVILLE,TN,37205,363LA2100X
7,Acute Care,295,24337,0.012121,1487788188,1.0,None,ROACH,ANDREW,NICHOLAS,None,None,ACNP,3601 THE VANDERBILT CLINIC,None,NASHVILLE,TN,37232,363LA2100X
8,Acute Care,279,24337,0.011464,1457628265,1.0,None,SCHALLENBERGER,KAITLYN,None,None,None,ACNP-BC,4230 HARDING PIKE,SUITE 503,NASHVILLE,TN,37205,363LA2100X
9,Acute Care,278,24337,0.011423,1720070949,1.0,None,BARR,CAREY,L.,None,None,ACNP-BC,854 W JAMES CAMPBELL BLVD,STE. 101A,COLUMBIA,TN,38401,363LA2100X


In [57]:
None_Vandy_byspecialization_sqlite[None_Vandy_byspecialization_sqlite['specialization'] == 'Family']

,specialization,total_patients,specialization_total_patients_ref,specialization_referral_ratio,npi,entity_type_code,provider_organization_name_(legal_business_name),provider_last_name_(legal_name),provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,provider_credential_text,provider_first_line_business_practice_location_address,provider_second_line_business_practice_location_address,provider_business_practice_location_address_city_name,provider_business_practice_location_address_state_name,provider_business_practice_location_address_postal_code,primary_taxonomy
422,Family,874,58974,0.014820,1750621579,1.0,None,DICKEY,TIMOTHY,CLAYTON,None,None,FNP-C,927 N JAMES CAMPBELL BLVD,SUITE 105,COLUMBIA,TN,38401,363LF0000X
423,Family,612,58974,0.010377,1013434935,1.0,None,DIMOND,JILLIAN,None,None,None,"MSN, FNP-BC",300 20TH AVE N STE G4,None,NASHVILLE,TN,37203,363LF0000X
424,Family,606,58974,0.010276,1477960680,1.0,None,GALBREATH,KERI,None,None,None,FNP-C,3441 DICKERSON PIKE,None,NASHVILLE,TN,37207,363LF0000X
425,Family,596,58974,0.010106,1134557481,1.0,None,DIEROLF,TINA,GAIL,MS.,None,APN-C,119 CUMBERLAND PL,None,HENDERSONVILLE,TN,37075,363LF0000X
426,Family,584,58974,0.009903,1265809305,1.0,None,CHAPMAN,LINDSEY,GAIL,MRS.,None,NP-C,1222 TROTWOOD AVE,SUITE 601,COLUMBIA,TN,38401,363LF0000X
427,Family,558,58974,0.009462,1710394705,1.0,None,BOWERS,PATRICIA,ANNE,MS.,None,MSN NP,300 20TH AVE N,SUITE G4,NASHVILLE,TN,37203,363LF0000X
428,Family,520,58974,0.008817,1194021352,1.0,None,FINE,LINDA,DARLENE,None,None,FNP-BC,3005 AMBROSE AVE,None,NASHVILLE,TN,37207,363LF0000X
429,Family,516,58974,0.008750,1659619161,1.0,None,BECK,DAVID,BRIAN,None,None,"FNP, NP-C",1222 TROTWOOD AVE STE 503,None,COLUMBIA,TN,38401,363LF0000X
430,Family,490,58974,0.008309,1316324593,1.0,None,HOPKINS,MEGAN,ELIZABETH,None,None,FNP-BC,2400 PATTERSON ST,None,NASHVILLE,TN,37203,363LF0000X
431,Family,477,58974,0.008088,1780961987,1.0,None,HARGROVE,SHELLY,MARIE,MRS.,None,NP-C,1090 N ELLINGTON PKWY STE 102,None,LEWISBURG,TN,37091,363LF0000X
